In [1]:
import pandas as pd
import torch
from transformers import pipeline


/Users/Juliepronier/Documents/02. Work/03. Search and Interviews/spotify-data-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test HuggingFace call


In [ ]:
# sentiment_test = pipeline(
#    "sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
# response = sentiment_test(
#    "I think my Spotify AI dj is learning that I'm autistic and only like listening to the songs I already have on my playlist"
# )

In [ ]:
# response

## Sentiment Analysis on collected tweets


In [2]:
tweets = pd.read_csv("data/spotify_ai_dj_all_tweets.csv")
tweets.head()

,id,text,author_id,created_at,likes,retweets,replies,text_clean,text_length
0,1961358481375518801,"@Spotify, can your AI DJ please stop sucking u...",1310212024253775872,2025-08-29 09:21:16+00:00,1,0,1,@user can your AI DJ please stop sucking up to...,142
1,1961360088419205571,@realfancy225 @Spotify @JonahKagen @ThisIsRobT...,1720665183188922368,2025-08-29 09:27:40+00:00,0,0,0,@user @user @user @user Totally agree—Spotify'...,262
2,1961387792279429319,@jaidenodavis @snseics1999 weird it’s exactly ...,1433597023576420365,2025-08-29 11:17:45+00:00,6,0,0,@user @user weird it’s exactly the “look” that...,305
3,1961390483772514589,🎧🤖 Meet your new AI DJ on Spotify — spinning t...,1936411644998893568,2025-08-29 11:28:26+00:00,0,0,0,🎧🤖 Meet your new AI DJ on Spotify — spinning t...,220
4,1961449992477970463,having the Spotify Ai DJ say my name to feel s...,747967587888271360,2025-08-29 15:24:54+00:00,1,0,1,having the Spotify Ai DJ say my name to feel s...,54


In [3]:
tweets.shape


(144, 9)

In [4]:
# Run sentiment analysis on the 'text_clean' column using the classifier and device selection above


# Only consider cuda or cpu, ignore mps


device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    classifier = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=0,
    )
else:
    classifier = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=-1,
    )


# Get both sentiment and confidence (score) for each tweet
def get_sentiment_and_confidence(text):
    result = classifier(text)[0]
    return pd.Series({"sentiment": result["label"], "confidence": result["score"]})


tweets[["sentiment", "confidence"]] = tweets["text_clean"].apply(
    get_sentiment_and_confidence
)
tweets.head()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


,id,text,author_id,created_at,likes,retweets,replies,text_clean,text_length,sentiment,confidence
0,1961358481375518801,"@Spotify, can your AI DJ please stop sucking u...",1310212024253775872,2025-08-29 09:21:16+00:00,1,0,1,@user can your AI DJ please stop sucking up to...,142,negative,0.906410
1,1961360088419205571,@realfancy225 @Spotify @JonahKagen @ThisIsRobT...,1720665183188922368,2025-08-29 09:27:40+00:00,0,0,0,@user @user @user @user Totally agree—Spotify'...,262,positive,0.923209
2,1961387792279429319,@jaidenodavis @snseics1999 weird it’s exactly ...,1433597023576420365,2025-08-29 11:17:45+00:00,6,0,0,@user @user weird it’s exactly the “look” that...,305,negative,0.827388
3,1961390483772514589,🎧🤖 Meet your new AI DJ on Spotify — spinning t...,1936411644998893568,2025-08-29 11:28:26+00:00,0,0,0,🎧🤖 Meet your new AI DJ on Spotify — spinning t...,220,positive,0.862370
4,1961449992477970463,having the Spotify Ai DJ say my name to feel s...,747967587888271360,2025-08-29 15:24:54+00:00,1,0,1,having the Spotify Ai DJ say my name to feel s...,54,neutral,0.794417


In [5]:
tweets.sentiment.value_counts()

sentiment
neutral     59
positive    54
negative    31
Name: count, dtype: int64

In [7]:
tweets.to_csv("data/processed/tweets_with_sentiment.csv", index=False)
